In [ ]:
from fastapi import FastAPI

import pandas as pd
import uvicorn

In [ ]:
app = FastAPI()
genreFunction = pd.read_parquet('./data/genre_functions.parquet')

In [ ]:
userRecommend = pd.read_parquet('./data/recommendations.parquet')

In [ ]:
items = pd.read_parquet('./data/items.parquet')
items

In [ ]:

# We apply the str.lower() method so that it does not matter how the word is written.
genreFunction['genres'] = genreFunction['genres'].str.lower()
genreFunction['genres'].unique()

In [ ]:
genreFunction

In [ ]:
mostPlayed = genreFunction.groupby('user_id')['playtime_forever'].sum()
mostPlayed.head()

In [ ]:
# The app.get decorator is used to associate the function with an HTTP GET request.
# '/PlayTimeGenre/{genre}/' is the path for the endpoint, the {genre} is the parameter.
@app.get('/PlayTimeGenre/{genre}/')
def PlayTimeGenre( genre : str ):
    '''
    Find the genre with the most playtime hours.

    Parameters
    ----------
    genre : str
        Desired genre to look for.
    
    Returns
    -------
    int
        Year when the highest number of hours played was recorded for that genre.
    
    Examples
    -------
    >>> PlayTimeGenre('Action')
    2012
    >>> PlayTimeGenre('Indie')
    2006

    '''
    # We look in our DataSet if there is any match with the genre entered.
    genres = genreFunction[genreFunction['genres'] == genre.lower()]

    # We calculate the total playtime for each year in the dataset.
    results = genres.groupby('year')['playtime_forever'].sum()
    
    # Locate the index where the maximum value is. In this case the most played genre.
    idResut = results.idxmax()

    
    
    if genres.empty:
        return f'The genre {genre} does not exist.'

    
    return f'Year with the most playtime hours for {genre}: {idResut}'

In [ ]:
test = PlayTimeGenre('RPG')
print(test)

# Segunda Función

In [ ]:
@app.get('/PlayTimeGenre/{genre}/')
def UserForGenre( genre : str ):
    '''
    Find the user with the most hours played by genre, as well as hours played for each year.

    Parameters
    ----------
    genre : str
        Desired genre to look for.
    
    Returns
    -------
    list
        The user name and the amount of hours played by year for that particular genre.
    
    Examples
    -------
    >>> UserForGenre('Simulation')
    UserName is the user with the most playtime for the genre "Action" with 23721 hours played.

    Year            2003    2006    2009    2010    2011    2012    2013    2014    2015    2016    
    Hours Played      0      0      2037    4102    1968     223     323    342     1224     112   
    '''
    
    
    # Look in the DataSet if there is any match with the genre entered.
    genres = genreFunction[genreFunction['genres'] == genre.lower()]

    if genres.empty:
        return f'The genre {genre} does not exist.'

    # Group the DataSet by user ID, then sum the amount of hours played.
    mostPlayed = genres.groupby('user_id')['playtime_forever'].sum().reset_index()
    
    # Locate the index for the player.
    player = genres.loc[genres['playtime_forever'].idxmax()]['user_id']

    # Filter the DataSet with only the player id
    filteredDFWithPlayerID = (genres[genres['user_id'] == player])

    # Create a new DF with just the year and playtime_forever columns
    hoursPlayedByYear = filteredDFWithPlayerID.groupby('year')['playtime_forever'].sum()
    
    
    print(f'{player} is the user with the most playtime for the genre "{genre.capitalize()}" with {filteredDFWithPlayerID["playtime_forever"].sum()} hours played.')
    print(f'\nYear\tHours Played\n')
    for year, hour in hoursPlayedByYear.items():
        print(f'{year}\t{hour}\n')
    


In [ ]:
UserForGenre('actiON')

# 3rd Function

In [262]:
userRecommend.head()

,title,id,user_id,playtime_forever,year,posted,item_id,recommend,sentiment_analysis
0,Carmageddon Max Pack,282010,UTNerd24,5,1997,"May 6, 2014.",244210,True,0
1,Carmageddon Max Pack,282010,UTNerd24,5,1997,"December 2, 2015.",440,True,0
2,Carmageddon Max Pack,282010,UTNerd24,5,1997,"June 6, 2015.",376920,True,2
3,Carmageddon Max Pack,282010,UTNerd24,5,1997,"September 29, 2015.",257750,True,1
4,Carmageddon Max Pack,282010,UTNerd24,5,1997,"September 8, 2014.",221910,True,0


In [323]:
@app.get('/UsersRecommend/{year}/')
def UsersRecommend( year : int ):
    '''
    Get the three most recommended games  
    
    Parameters
    ----------
    year : str
        Year in which the top three recommended games are.
    
    Returns
    -------
    list
        Name of the three recommended games.
    
    Examples
    -------
    >>> UsersRecommend(2018):
    Counter-Strike: Global Offense, Garry's Mode, Fall Guys
    >>> UsersRecommend(2021)
    Empire: Total War, Left 4 Dead 2, The Stanley Parable
    '''


    givenYear = userRecommend[userRecommend['year'] == year]

    if givenYear.empty:
        return f'The year {year} does not exist.'

    # Group the top three games
    topThree = (givenYear['title'].value_counts().head(3).reset_index()
        .rename(columns={ 'title': 'Game', 'count': 'Positive Reviews'}))
    # print((topThree))

    return [{f'Top {i+1}: "{game}" with {reviews} positive reviews'} for i, (game, reviews) in topThree.iterrows()]


In [324]:
UsersRecommend(2011)

[{'Top 1: "Terraria" with 31329 positive reviews'},
 {'Top 2: "Portal 2" with 26975 positive reviews'},
 {'Top 3: "PAYDAY™ The Heist" with 19889 positive reviews'}]

# 4th Function

In [ ]:
@app.get('/UsersNotRecommend/{year}/')
def UsersNotRecommend( year : int ):
    '''
    Get the three least recommended games.
    
    Parameters
    ----------
    year : str
        Year in which the three least recommended games are.
    
    Returns
    -------
    list
        Name of the three recommended games.
    
    Examples
    -------
    >>> UsersRecommend(2018):
    Counter-Strike: Global Offense, Garry's Mode, Fall Guys
    >>> UsersRecommend(2021)
    Empire: Total War, Left 4 Dead 2, The Stanley Parable
    '''

    pass